In [39]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

df = pd.read_csv(f"{os.path.dirname(os.path.abspath('__file__'))}\\4_insurance_data.csv")
print(df.head())
df.shape

GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
   age  affordibility  bought_insurance
0   22              1                 0
1   25              0                 0
2   47              1                 1
3   52              0                 0
4   46              1                 1


(28, 3)

In [40]:
# split train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

# apply scaling
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [41]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

1
Epoch 4803/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4804/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4805/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4806/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4807/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4808/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4809/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4810/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4811/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4667 - accuracy: 0.9091
Epoch 4812/5000
1/1 [==============================] - 0s 2ms/step - loss: 0.4667 - accur

In [42]:
model.evaluate(X_test_scaled, y_test) # perfect model

1/1 [==============================] - 0s 72ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

In [43]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.060868],
        [1.40865 ]], dtype=float32),
 array([-2.913703], dtype=float32))

In [44]:

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# manual prediction funcion using sigmoid
def prediction_function(age, affordibility):
    return sigmoid(coef[0] * age + coef[1] * affordibility + intercept)

def log_loss(y_actual, y_predicted):
    eplison = 1e-15
    y_predicted_new = np.array([min(max(i, eplison), 1 - eplison) for i in y_predicted])

    return -np.mean(y_actual * np.log(y_predicted_new) + (1 - y_actual) * np.log(1 - y_predicted_new))

In [45]:
print(prediction_function(0.47, 1))
print(model.predict(X_test_scaled))

[0.7054848]
[[0.7054848 ]
 [0.35569546]
 [0.16827849]
 [0.47801176]
 [0.7260697 ]
 [0.8294984 ]]


In [54]:
# custom (own) model
class DNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X["age"], X["affordibility"], y, epochs, loss_threshold)
    
    def predict(self, X_test):
        return sigmoid(self.w1 * X_test["age"] + self.w2 * X_test["affordibility"] + self.bias)

    def gradient_descent(self, age, affordibility, y_actual, epochs, loss_threshold):
        w1 = w2 = 1
        bias, rate, n = 0, 0.5, len(age)

        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordibility + bias

            # predict and calculate loss
            y_predicted = sigmoid(weighted_sum)
            loss = log_loss(y_actual, y_predicted)

            # use gradient descent to calculate weighted
            w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_actual))
            w2d = (1/n) * np.dot(np.transpose(affordibility), (y_predicted - y_actual))

            bias_d = np.mean(y_predicted - y_actual)

            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d

            if i % 50 == 0:
                print(f"Epoch: {i}/{epochs} - w1: {w1} - w2: {w2} - bias: {bias} - loss: {loss})")

            if loss <= loss_threshold:
                print(f"Epoch: {i}/{epochs} - w1: {w1} - w2: {w2} - bias: {bias} - loss: {loss})")
                break

        return w1, w2, bias

model_1 = DNN()
model_1.fit(X_train_scaled, y_train, epochs=500, loss_threshold=0.4631)

Epoch: 0/500 - w1: 0.974907633470177 - w2: 0.948348125394529 - bias: -0.11341867736368583 - loss: 0.7113403233723417)
Epoch: 50/500 - w1: 1.5033195541731388 - w2: 1.108384790367645 - bias: -1.2319047301235464 - loss: 0.5675865113475955)
Epoch: 100/500 - w1: 2.200713131760032 - w2: 1.2941584023238903 - bias: -1.6607009122062801 - loss: 0.5390680417774752)
Epoch: 150/500 - w1: 2.8495727769689085 - w2: 1.3696895491572745 - bias: -1.986105845859897 - loss: 0.5176462164249294)
Epoch: 200/500 - w1: 3.443016970881803 - w2: 1.4042218624465033 - bias: -2.2571369883752723 - loss: 0.5005011269691375)
Epoch: 250/500 - w1: 3.982450494649576 - w2: 1.4239127329321233 - bias: -2.494377365971801 - loss: 0.48654089537617085)
Epoch: 300/500 - w1: 4.472179522095915 - w2: 1.438787986553552 - bias: -2.707387811922373 - loss: 0.4750814640632793)
Epoch: 350/500 - w1: 4.917245868007634 - w2: 1.4525660781176122 - bias: -2.901176333556766 - loss: 0.46561475306999006)
Epoch: 366/500 - w1: 5.051047623653049 - w2: 

In [55]:
# compare results between our own model vs tensorflow model (very similar)
print(model_1.predict(X_test_scaled))
print(model.predict(X_test_scaled))

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64
[[0.7054848 ]
 [0.35569546]
 [0.16827849]
 [0.47801176]
 [0.7260697 ]
 [0.8294984 ]]
